In [2]:
# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [1]:
# !python -m bitsandbytes

In [3]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Fri Mar 21 04:17:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0       

In [4]:
!pip install bitsandbytes

In [5]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [6]:
import torch
import bitsandbytes as bnb

print("CUDA available:", torch.cuda.is_available())
print("Torch version:", torch.__version__)
print("Bitsandbytes version:", bnb.__version__)

CUDA available: True
Torch version: 2.6.0+cu124
Bitsandbytes version: 0.45.3


In [11]:
# import IPython
# IPython.display.clear_output()

In [7]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Data Processing

In [8]:
dataset = load_dataset("wikisql", trust_remote_code=True)
dataset

README.md:   0%|          | 0.00/7.80k [00:00<?, ?B/s]

wikisql.py:   0%|          | 0.00/6.57k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 15878
    })
    validation: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 8421
    })
    train: Dataset({
        features: ['phase', 'question', 'table', 'sql'],
        num_rows: 56355
    })
})

### Tokenisation

In [9]:
model_name =  "facebook/bart-base"
# instantiate tokenizer of llama2
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
# test the functionality of the tokenizer

question = dataset['train'][0]['question']
print('Question:', question)

tokenized_example = tokenizer(dataset['train'][0]['question'])
print('After tokenisation:', tokenized_example)

Question: Tell me what the notes are for South Australia 
After tokenisation: {'input_ids': [0, 35438, 162, 99, 5, 2775, 32, 13, 391, 1221, 1437, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [11]:
# finding the max length for tokenization
tokenizer.model_max_length

1024

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = []
    for question, table in zip(examples['question'], examples['table']):
        table_str = " | ".join([f"{header} ({type})" for header, type in zip(table['header'], table['types'])])
        input_str = f"Convert the Question to SQL: {question}, based on the table: {table_str}"
        inputs.append(input_str)

    targets = [sql['human_readable'] for sql in examples['sql']]

    ### tokenise the input (prompt) and answers (sql queries)
    model_inputs = tokenizer(inputs, padding='max_length', max_length=512, truncation=True, return_tensors='pt')
    labels = tokenizer(targets, padding='max_length', max_length=128, truncation=True, return_tensors='pt')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [13]:
train_set = dataset['train'].map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/56355 [00:00<?, ? examples/s]

In [14]:
validation_set = dataset['validation'].map(preprocess_function, batched=True, remove_columns=dataset['validation'].column_names)

Map:   0%|          | 0/8421 [00:00<?, ? examples/s]

In [15]:
test_set = dataset['test'].map(preprocess_function, batched=True, remove_columns=dataset['test'].column_names)

Map:   0%|          | 0/15878 [00:00<?, ? examples/s]

# Model Training

### Evaluation Metric

In [16]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.0 MB/s eta 0:00:00


In [17]:
from evaluate import load
bleu_metric = load("bleu")

In [18]:
def compute_metrics(pred):
    # Retrive predicted tokens
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # Decode the tokens and convert to text
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id # n the labels_ids array, the value -100 represents ignored tokens (often used during loss computation to ignore padding). It is replaced with the padding token ID (tokenizer.pad_token_id) to avoid errors during decoding.
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # Compute BLEU score
    '''
    BLEU Metric:

    Measures how closely predicted text matches the reference text.
    Compares n-grams in predictions with those in references.

    Input Format:

    predictions: A list of tokenized predicted strings (pred_str split into words).
    references: A list of lists of tokenized reference strings (label_str split into words).
    Each reference is nested in a list to support multiple references per prediction.
    '''

    bleu = bleu_metric.compute(predictions=pred_str, references=label_str)
    # bleu = bleu_metric.compute(predictions=[p.split() for p in pred_str], references=[[l.split()] for l in label_str])

    return {"bleu": bleu["bleu"]}

### Model Parameters

In [19]:
# DataCollatorForSeq2Seq dynamically pads the sentences to the longest length in a batch during
# collation, instead of padding the whole dataset to the maximum length.
# These elements are of the same type as the elements of train_dataset or eval_dataset.

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name, padding=True, return_tensors='pt')

In [20]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "facebook/bart-base"
# instantiate tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

training_args = Seq2SeqTrainingArguments(
    output_dir="./bart_baseline_results1",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    fp16=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

### Results on Baseline Model (Before Fine Tuning)

In [21]:
trainer.evaluate()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mlohakar (mlohakar-university-of-southern-california) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 16.303125381469727,
 'eval_bleu': 0.02268940428374036,
 'eval_runtime': 618.3882,
 'eval_samples_per_second': 25.676,
 'eval_steps_per_second': 1.606}

In [25]:
import pandas as pd

# Select some samples
sample_range = range(20)
test_examples = test_set.select(sample_range)

# Make predictions
pred = trainer.predict(test_examples)
# Retrive predicted tokens
pred_ids = pred.predictions

# Decode the prediction
pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

# Create a DataFrame
questions = [example['question'] for example in dataset['test'].select(sample_range)]
header = [example['table']['header'] for example in dataset['test'].select(sample_range)]
labels = [example['sql']['human_readable'] for example in dataset['test'].select(sample_range)]

df = pd.DataFrame({
    "Question": questions,
    "Header": header,
    "Prediction": pred_str,
    "Target": labels
})

# Show all columns & rows
pd.set_option('display.max_colwidth', None)
df.head(10)

,Question,Header,Prediction,Target
0,What is terrence ross' nationality,"[Player, No., Nationality, Position, Years in Toronto, School/Club Team]","Convert the Question to SQL: What is terrence ross' nationality, based",SELECT Nationality FROM table WHERE Player = Terrence Ross
1,What clu was in toronto 1995-96,"[Player, No., Nationality, Position, Years in Toronto, School/Club Team]",Convert the Question to SQL: What clu was in toronto 1995-96,SELECT School/Club Team FROM table WHERE Years in Toronto = 1995-96
2,which club was in toronto 2003-06,"[Player, No., Nationality, Position, Years in Toronto, School/Club Team]","Convert the Question to SQL: which club was in toronto 2003-06,",SELECT School/Club Team FROM table WHERE Years in Toronto = 2003-06
3,how many schools or teams had jalen rose,"[Player, No., Nationality, Position, Years in Toronto, School/Club Team]",Convert the Question to SQL: how many schools or teams had jalen rose,SELECT COUNT School/Club Team FROM table WHERE Player = Jalen Rose
4,Where was Assen held?,"[No, Date, Round, Circuit, Pole Position, Fastest Lap, Race winner, Report]","Convert the Question to SQL: Where was Assen held?, based on the table",SELECT Round FROM table WHERE Circuit = Assen
5,What was the number of race that Kevin Curtain won?,"[No, Date, Round, Circuit, Pole Position, Fastest Lap, Race winner, Report]",Convert the Question to SQL: What was the number of race that Kevin Curtain,SELECT COUNT No FROM table WHERE Pole Position = Kevin Curtain
6,What was the date of the race in Misano?,"[No, Date, Round, Circuit, Pole Position, Fastest Lap, Race winner, Report]",Convert the Question to SQL: What was the date of the race in Misano,SELECT Date FROM table WHERE Circuit = Misano
7,How many different positions did Sherbrooke Faucons (qmjhl) provide in the draft?,"[Pick, Player, Position, Nationality, NHL team, College/junior/club team]",Convert the Question to SQL: How many different positions did Sherbrooke Fau,SELECT COUNT Position FROM table WHERE College/junior/club team = Sherbrooke Faucons (QMJHL)
8,What are the nationalities of the player picked from Thunder Bay Flyers (ushl),"[Pick, Player, Position, Nationality, NHL team, College/junior/club team]",Convert the Question to SQL: What are the nationalities of the player picked from,SELECT Nationality FROM table WHERE College/junior/club team = Thunder Bay Flyers (USHL)
9,How many different college/junior/club teams provided a player to the Washington Capitals NHL Team?,"[Pick, Player, Position, Nationality, NHL team, College/junior/club team]",Convert the Question to SQL: How many different college/junior/club teams,SELECT COUNT College/junior/club team FROM table WHERE NHL team = Washington Capitals


### Model Fine Tuning

In [26]:
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 442,368 || all params: 139,862,784 || trainable%: 0.3162871404018384


In [27]:
# hyperparameters in the training_args are cited from https://github.com/anyuanay/medium/blob/main/src/working_huggingface/Working_with_HuggingFace_ch3_Fine_Tuning_T5_Small_Text_Summarization_Model.ipynb
training_args = Seq2SeqTrainingArguments(
    output_dir="bart_wikisql",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=validation_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu
1,4.847700,4.261711,0.700413
2,4.810500,4.244424,0.744639
3,4.794900,4.240922,0.754982


TrainOutput(global_step=10569, training_loss=4.979721968780122, metrics={'train_runtime': 2208.4556, 'train_samples_per_second': 76.553, 'train_steps_per_second': 4.786, 'total_flos': 5.177229879803904e+16, 'train_loss': 4.979721968780122, 'epoch': 3.0})

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
# mount gdrive
from google.colab import drive
drive.mount('/content/drive')

# Save the model
model_folder_path = "/content/drive/MyDrive/bart_finetuned_wikisql"

model.save_pretrained(model_folder_path)
tokenizer.save_pretrained(model_folder_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/MyDrive/bart_finetuned_wikisql/tokenizer_config.json',
 '/content/drive/MyDrive/bart_finetuned_wikisql/special_tokens_map.json',
 '/content/drive/MyDrive/bart_finetuned_wikisql/vocab.json',
 '/content/drive/MyDrive/bart_finetuned_wikisql/merges.txt',
 '/content/drive/MyDrive/bart_finetuned_wikisql/added_tokens.json',
 '/content/drive/MyDrive/bart_finetuned_wikisql/tokenizer.json')

# Evaluating after Fine Tuning

In [36]:
# load fine-tuned model
model_folder_path = "/content/drive/MyDrive/bart_finetuned_wikisql"
model = AutoModelForSeq2SeqLM.from_pretrained(model_folder_path)
tokenizer = AutoTokenizer.from_pretrained(model_folder_path)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

OSError: /content/drive/MyDrive/bart_finetuned_wikisql does not appear to have a file named config.json. Checkout 'https://huggingface.co//content/drive/MyDrive/bart_finetuned_wikisql/None' for available files.

In [37]:
trainer.evaluate()

{'eval_loss': 4.240922451019287,
 'eval_bleu': 0.7549823645428032,
 'eval_runtime': 324.8097,
 'eval_samples_per_second': 25.926,
 'eval_steps_per_second': 1.622,
 'epoch': 3.0}

In [38]:
test_examples = test_set.select(sample_range)

# Make predictions
pred = trainer.predict(test_examples)
# Retrive predicted tokens
pred_ids = pred.predictions

# Decode the prediction
pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

# Create a DataFrame
questions = [example['question'] for example in dataset['test'].select(sample_range)]
header = [example['table']['header'] for example in dataset['test'].select(sample_range)]
labels = [example['sql']['human_readable'] for example in dataset['test'].select(sample_range)]

df = pd.DataFrame({
    "Question": questions,
    "Header": header,
    "Prediction": pred_str,
    "Target": labels
})

# Show all columns & rows
pd.set_option('display.max_colwidth', None)
df.head(10)

,Question,Header,Prediction,Target
0,What is terrence ross' nationality,"[Player, No., Nationality, Position, Years in Toronto, School/Club Team]",SELECT Nationality FROM table WHERE Player = terrence ross',SELECT Nationality FROM table WHERE Player = Terrence Ross
1,What clu was in toronto 1995-96,"[Player, No., Nationality, Position, Years in Toronto, School/Club Team]",SELECT Player FROM table WHERE Years in Toronto = 1995-96,SELECT School/Club Team FROM table WHERE Years in Toronto = 1995-96
2,which club was in toronto 2003-06,"[Player, No., Nationality, Position, Years in Toronto, School/Club Team]",SELECT School/Club Team FROM table WHERE Years in Toronto = 2003-06,SELECT School/Club Team FROM table WHERE Years in Toronto = 2003-06
3,how many schools or teams had jalen rose,"[Player, No., Nationality, Position, Years in Toronto, School/Club Team]",SELECT COUNT School/Club Team FROM table WHERE Player = jalen rose,SELECT COUNT School/Club Team FROM table WHERE Player = Jalen Rose
4,Where was Assen held?,"[No, Date, Round, Circuit, Pole Position, Fastest Lap, Race winner, Report]",SELECT No FROM table WHERE Circuit = Assen,SELECT Round FROM table WHERE Circuit = Assen
5,What was the number of race that Kevin Curtain won?,"[No, Date, Round, Circuit, Pole Position, Fastest Lap, Race winner, Report]",SELECT MAX Race winner FROM table WHERE No = Kevin Curtain,SELECT COUNT No FROM table WHERE Pole Position = Kevin Curtain
6,What was the date of the race in Misano?,"[No, Date, Round, Circuit, Pole Position, Fastest Lap, Race winner, Report]",SELECT Date FROM table WHERE Circuit = misano,SELECT Date FROM table WHERE Circuit = Misano
7,How many different positions did Sherbrooke Faucons (qmjhl) provide in the draft?,"[Pick, Player, Position, Nationality, NHL team, College/junior/club team]",SELECT COUNT Position FROM table WHERE Player = sherbrooke faucons (q,SELECT COUNT Position FROM table WHERE College/junior/club team = Sherbrooke Faucons (QMJHL)
8,What are the nationalities of the player picked from Thunder Bay Flyers (ushl),"[Pick, Player, Position, Nationality, NHL team, College/junior/club team]",SELECT Nationality FROM table WHERE NHL team = Thunder Bay Flyers (ushl),SELECT Nationality FROM table WHERE College/junior/club team = Thunder Bay Flyers (USHL)
9,How many different college/junior/club teams provided a player to the Washington Capitals NHL Team?,"[Pick, Player, Position, Nationality, NHL team, College/junior/club team]",SELECT COUNT College/junior/club team FROM table WHERE NHL team = Washington,SELECT COUNT College/junior/club team FROM table WHERE NHL team = Washington Capitals
